1D CNN model using arrays of length 20,000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import pickle

file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset_compressed.pkl.gz'

with gzip.open(file_path, 'rb') as f:
    test_dataset = pickle.load(f)

In [ ]:
import gzip
import pickle

file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/train_dataset.pkl'

with open(file_path, 'rb') as f:
    train_dataset = pickle.load(f)

In [ ]:
all_audio_data = np.concatenate(train_dataset['number array'])

min_value = np.min(all_audio_data)
max_value = np.max(all_audio_data)
mean_value = np.mean(all_audio_data)
print(min_value)
print(max_value)
print(mean_value)

-1.629381
1.7866867
-9.760262e-05


In [ ]:
train_dataset['number array'][3]

array([-6.4919252e-09, -5.2655702e-09,  5.3367422e-09, ...,
        6.5053061e-09, -8.9570982e-09,  3.3688035e-09], dtype=float32)

In [ ]:
import numpy as np

audio_lengths = []
for i in range(len(train_dataset['number array'])):
    audio_lengths.append(len(train_dataset['number array'][i]))

max_length = max(audio_lengths)
print("Maximum length:", max_length)

average_length = np.mean(audio_lengths)
print(f"The average length of the arrays is: {average_length}")

padding_length = int(np.percentile(audio_lengths, 70))
print(f"Padding length: {padding_length}")

Maximum length: 170112
The average length of the arrays is: 86812.092375
Padding length: 101376


In [ ]:
new_arrays = []

all_audio_data = np.concatenate(train_dataset['number array'])

min_value = np.min(all_audio_data)
max_value = np.max(all_audio_data)

for i in range(len(train_dataset['number array'])):
  y = train_dataset['number array'][i]
  target_length = 20000 # using avg length for now
  if len(y) < target_length:
    y = np.pad(y, (0, target_length - len(y)), 'constant') # Pad the audio with zeros if it's shorter than the target length
  else:
    y = y[:target_length] # Trim the audio if it's longer than the target length

  # normalizing the arrays range [0,1]
  #y_normalized = (y - min_value) / (max_value - min_value)
  new_arrays.append(y)

train_dataset.drop('number array', axis=1, inplace=True)
train_dataset['number array'] = new_arrays

In [ ]:
len(train_dataset['number array'][1])

20000

In [ ]:
new_arrays = []

all_audio_data = np.concatenate(test_dataset['number array'])

min_value = np.min(all_audio_data)
max_value = np.max(all_audio_data)

for i in range(len(test_dataset['number array'])):
  y = test_dataset['number array'][i]
  target_length = 20000 # using avg length for now
  if len(y) < target_length:
    y = np.pad(y, (0, target_length - len(y)), 'constant') # Pad the audio with zeros if it's shorter than the target length
  else:
    y = y[:target_length] # Trim the audio if it's longer than the target length

  # normalizing the arrays range [0,1]
  #y_normalized = (y - min_value) / (max_value - min_value)
  new_arrays.append(y)

test_dataset.drop('number array', axis=1, inplace=True)
test_dataset['number array'] = new_arrays

In [ ]:
X_train = np.array(train_dataset['number array'].to_list())
y_train = train_dataset['label']

In [ ]:
X_test = np.array(test_dataset['number array'].to_list())
y_test = test_dataset['label']

In [ ]:
len(test_dataset['number array'][0])

90000

In [ ]:
X_train.shape

(8000, 90000)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20000, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the 3D output to 1D for the dense layer
model.add(Flatten())

# Add a fully connected (dense) layer
model.add(Dense(64, activation='relu'))

# Add the output layer with a sigmoid activation (binary classification)
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Reshaping the data to add a channel dimension (required for CNN)
X_train = X_train.reshape(-1, 20000, 1)  # Adding the channel dimension

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 19998, 64)           │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 9999, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 9997, 128)           │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 4998, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 639744)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      40,943,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,968,705 (156.28 MB)

 Trainable params: 40,968,705 (156.28 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.9943 - loss: 0.0184 - val_accuracy: 0.9994 - val_loss: 0.0035
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - accuracy: 0.9994 - loss: 0.0025 - val_accuracy: 0.9950 - val_loss: 0.0143
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 102ms/step - accuracy: 1.0000 - loss: 8.7335e-04 - val_accuracy: 0.9962 - val_loss: 0.0088
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 1.0000 - loss: 1.0283e-04 - val_accuracy: 0.9962 - val_loss: 0.0138
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 1.0000 - loss: 6.6413e-05 - val_accuracy: 0.9956 - val_loss: 0.0154
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 1.0000 - loss: 5.4792e-04 - val_accuracy: 0.9956 - val_loss: 0.0158
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - accuracy: 1.0000 - loss: 2.4299e-05 - val_accuracy: 0.9950 - val_loss: 0.0163
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 104ms/step - accura

In [ ]:
X_test = X_test.reshape(-1, 20000, 1)  # changing the dimensions to what the model can take in

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

ValueError: Cannot take the length of shape with unknown rank.

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Thresholding to get binary predictions (0 or 1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate key metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
fpr = fp / (fp + tn)  # False positive rate
specificity = tn / (tn + fp)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"F1-Score: {f1}")
print(f"False Positive Rate: {fpr}")
print(f"Specificity: {specificity}")

Accuracy: 0.989
Precision: 0.9543269230769231
Recall (Sensitivity): 0.9925
F1-Score: 0.9730392156862745
False Positive Rate: 0.011875
Specificity: 0.988125


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Predict probabilities for the positive class
y_pred_prob = model.predict(X_test)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred_prob)
print("AUC Score:", auc_score)

In [ ]:
# just testing

import numpy as np

# Example: Assuming you have a dataset prepared with numpy arrays
X_train = np.random.rand(8000, 20000, 1)  # Random data for testing purposes
y_train = np.random.randint(0, 2, 8000)  # Random binary labels for testing purposes
X_test = np.random.rand(2000, 20000, 1)  # Test data
y_test = np.random.randint(0, 2, 2000)   # Test labels


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(20000, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")


Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 20s 89ms/step - accuracy: 0.5038 - loss: 1.7065 - val_accuracy: 0.5219 - val_loss: 0.6930
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - accuracy: 0.5031 - loss: 0.6932 - val_accuracy: 0.5219 - val_loss: 0.6930
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.5060 - loss: 0.6931 - val_accuracy: 0.5219 - val_loss: 0.6931
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 21s 88ms/step - accuracy: 0.4963 - loss: 0.6932 - val_accuracy: 0.5219 - val_loss: 0.6931
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.4997 - loss: 0.6932 - val_accuracy: 0.5219 - val_loss: 0.6931
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 22s 90ms/step - accuracy: 0.4892 - loss: 0.6932 - val_accuracy: 0.5219 - val_loss: 0.6931
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 20s 84ms/step - accuracy: 0.4870 - loss: 0.6932 - val_accuracy: 0.4781 - val_loss: 0.6932
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 21s 86ms/step - accuracy: 0.5053 - loss: 0.6931 - 